In [5]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import math 

PROJECT = 'qwiklabs-gcp-aebfb78fe0f1b1d1' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'sarah-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

SEQ_LEN = 50

In [6]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

In [7]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [8]:
! pwd
! ls

/content/datalab/energy_forcasing/Sarah
BIG_q.ipynb		CNN_W2P_model  Energy_ts       train-1.csv  valid-1.csv
Build_model_task.ipynb	data	       sarah_data.csv  trained


In [9]:
## move the csv file to the bucket ~ you rly dont need to do this
!gsutil cp -p /content/datalab/energy_forcasing/Sarah/sarah_data.csv gs://sarah-bucket/sarah_data.csv

Copying file:///content/datalab/energy_forcasing/Sarah/sarah_data.csv [Content-Type=text/csv]...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


In [6]:
CSV_COLUMNS = ['prediction_date', 'wind_speed_100m', 
  'wind_direction_100m', 
  'temperature', 
  'air_density',
  'pressure', 'precipitation', 'wind_gust', 'radiation', 
  'wind_speed', 'wind_direction', 'price']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df5 = pd.read_csv('/content/datalab/energy_forcasing/Sarah/sarah_data.csv', header = None, names = CSV_COLUMNS)
df5.describe()

,wind_speed_100m,wind_direction_100m,temperature,air_density,pressure,precipitation,wind_gust,radiation,wind_speed,wind_direction,price
count,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000,6646.000000
mean,4.799901,189.582975,14.920355,1.132338,935.286695,0.117364,4.872509,225.250182,3.220986,189.082629,50.153506
std,1.964154,58.126639,9.626527,0.039333,6.105083,0.245561,2.428587,266.299815,1.319704,56.712796,12.448392
min,1.372222,47.883333,-6.044444,1.036111,913.800000,0.000000,1.194444,0.000000,0.855556,50.538889,4.000000
25%,3.534722,142.598611,7.540278,1.106111,932.034722,0.000000,3.284722,0.000000,2.294444,143.750000,43.000000
50%,4.477778,191.908333,14.213889,1.131667,934.761111,0.000000,4.327778,95.000000,2.961111,191.208333,50.890000
75%,5.544444,232.043056,21.062500,1.160556,937.805556,0.100000,5.627778,416.422222,3.850000,230.865278,59.950000
max,15.044444,329.544444,41.044444,1.229444,961.238889,1.700000,16.705556,954.288889,9.738889,327.711111,85.050000


In [7]:
df5.shape

(6646, 12)

In [8]:
#for ii in range(10):
#   N = 7
#  SEQ = df5.price[(0+N*ii):(N+N*ii)]
np.random.seed(1) 
# np.random.uniform(0,6646-N)

In [9]:
# int(np.random.uniform(0,6646-N),0)

In [10]:
SEQ_LEN = 24 # Sequence of 1 week
with open('./data/timeserie_price_daily_train.csv', 'w') as f:
    to_csv = ''
    for idx, p in enumerate(df5['price']):
        to_csv += str(p) + ','
        if (idx + 1) % SEQ_LEN == 0:
            f.write(to_csv[:-1] + '\n')
            to_csv = ''

In [11]:
#int(round(np.random.uniform(0,6646-N),0))
#6646/50

In [12]:
def to_csv(filename, N, J):
  with open(filename, 'w') as ofp:
    start = 0
    end = J
    for ii in xrange(0, N):
      nn = int(round(np.random.uniform(low = 0, high = 6466-N),0))
      #print((nn),(nn+J))
      seq = df5.price[(nn):(nn+J)]
      #seq = df5.price[start: end]
      #start += STRIDE
      #end += STRIDE
      line = ",".join(map(str, seq))
      ofp.write(line + '\n')
      #print(len(seq))
      if len(seq) != J:
        break

import os
try:
  os.makedirs('data/CNN_W2P_model/')
except OSError:
  pass
to_csv('data/CNN_W2P_model/train-1.csv', N = 100, J = 50)  # 1000 sequences
to_csv('data/CNN_W2P_model/valid-1.csv', N = 150, J = 50)

In [13]:
df2 = pd.read_csv('data/CNN_W2P_model/train-1.csv', header = None)
df2.head()
#df2.describe()
# df2.count()
df2.shape

(100, 50)

In [14]:
msk = np.random.rand(len(df2)) < 0.8
train = df2[msk]
test = df2[~msk]
train
# then normalize
# then write to train.csv, test.csv df2.to_csv(...)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,39.50,39.75,67.10,67.10,66.20,65.51,50.48,59.23,28.30,38.40,...,63.50,68.10,50.84,64.10,39.41,45.10,46.0,53.74,69.10,43.88
2,56.55,54.74,30.00,46.86,39.25,48.50,43.49,59.57,40.13,45.00,...,30.00,65.10,57.20,30.00,50.76,47.08,45.1,14.84,60.10,60.50
4,19.99,59.85,41.21,39.50,39.82,60.10,38.40,52.10,51.90,50.78,...,68.52,4.00,34.77,50.85,54.04,55.33,57.9,61.00,41.38,44.00
6,57.80,54.32,59.10,60.10,60.00,64.10,57.98,36.10,48.10,52.64,...,50.30,64.03,48.96,58.30,50.10,63.52,67.1,47.38,62.93,71.00
7,60.83,60.43,35.50,50.72,36.66,49.00,42.60,64.00,52.45,6.33,...,32.01,28.50,42.94,50.20,43.98,55.07,63.0,38.50,65.01,67.52


In [15]:
df2 = pd.read_csv('data/CNN_W2P_model/valid-1.csv', header = None) #, names = CSV_COLUMNS)
df2.describe()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.00000,150.000000,150.000000,150.000000,150.000000,...,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.00000,150.000000,150.000000
mean,49.451000,49.661800,50.469200,49.572733,48.333333,49.72480,50.167533,49.962933,50.616200,50.582533,...,49.260800,50.301000,48.182733,50.816600,50.154667,48.505667,49.065933,49.84940,49.457000,49.530733
std,13.584529,12.427447,12.939042,12.791665,11.614033,12.46928,11.895658,12.916401,13.154818,13.038634,...,12.507019,11.998725,12.191564,12.511147,12.758544,13.445205,12.239636,12.15816,13.310002,14.457003
min,4.800000,12.400000,5.000000,5.000000,5.000000,4.99000,10.000000,4.990000,8.400000,5.000000,...,4.000000,4.990000,12.400000,13.000000,7.000000,5.000000,12.400000,14.35000,5.000000,4.800000
25%,41.422500,43.770000,41.725000,41.655000,40.880000,42.59000,43.100000,43.815000,43.665000,43.172500,...,42.497500,43.000000,42.025000,43.470000,42.710000,43.280000,42.812500,42.11750,42.000000,43.430000
50%,50.735000,50.010000,50.240000,51.175000,47.650000,51.34500,50.000000,51.665000,52.450000,53.035000,...,50.995000,49.700000,48.910000,51.440000,51.270000,48.810000,50.310000,50.00000,49.930000,50.490000
75%,59.890000,58.877500,61.442500,59.947500,57.197500,58.97500,59.425000,59.595000,60.100000,59.750000,...,58.892500,60.527500,56.642500,60.675000,60.075000,58.980000,57.857500,59.00500,59.847500,59.697500
max,78.900000,71.750000,70.300000,70.100000,72.520000,71.15000,73.000000,77.510000,71.000000,73.600000,...,70.570000,78.900000,69.980000,73.000000,70.400000,85.050000,72.210000,77.15000,71.670000,75.000000


In [16]:
%%bash
rm -rf /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
mkdir /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer
touch /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/__init__.py

In [17]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py
#!/usr/bin/env python

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

TIMESERIES_COL = 'price'
N_OUTPUTS = 1  # in each sequence, 1-49 are features, and 50 is label
SEQ_LEN = None
DEFAULTS = None
N_INPUTS = None


def init(hparams):
    global SEQ_LEN, DEFAULTS, N_INPUTS
    SEQ_LEN = hparams['sequence_length']
    DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
    N_INPUTS = SEQ_LEN - N_OUTPUTS


def linear_model(features, mode, params):
    X = features[TIMESERIES_COL]
    predictions = tf.layers.dense(X, 1, activation=None)
    return predictions


def dnn_model(features, mode, params):
    X = features[TIMESERIES_COL]
    h1 = tf.layers.dense(X, 10, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # linear output: regression
    return predictions


def cnn_model(features, mode, params):
    X = tf.reshape(features[TIMESERIES_COL],
                   [-1, N_INPUTS, 1])  # as a 1D "sequence" with only one time-series observation (height)
    c1 = tf.layers.conv1d(X, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p1 = tf.layers.max_pooling1d(c1, pool_size=2, strides=2)

    c2 = tf.layers.conv1d(p1, filters=N_INPUTS // 2,
                          kernel_size=3, strides=1,
                          padding='same', activation=tf.nn.relu)
    p2 = tf.layers.max_pooling1d(c2, pool_size=2, strides=2)

    outlen = p2.shape[1] * p2.shape[2]
    c2flat = tf.reshape(p2, [-1, outlen])
    h1 = tf.layers.dense(c2flat, 3, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # linear output: regression
    return predictions


def rnn_model(features, mode, params):
    CELL_SIZE = N_INPUTS // 3  # size of the internal state in each of the cells

    # 1. dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell = tf.nn.rnn_cell.GRUCell(CELL_SIZE)
    outputs, state = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)

    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h1 = tf.layers.dropout(inputs=h1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h2 = tf.layers.dense(dropout_h1, N_INPUTS // 2, activation=tf.nn.relu6)
    h3 = tf.layers.dense(h2, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h3 = tf.layers.dropout(inputs=h3, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h4 = tf.layers.dense(dropout_h3, N_INPUTS // 2, activation=tf.nn.relu6)
    h5 = tf.layers.dense(h4, N_INPUTS // 2, activation=tf.nn.relu)
    h6 = tf.layers.dense(h5, N_INPUTS // 2, activation=tf.nn.relu6)
    h7 = tf.layers.dense(h6, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h7 = tf.layers.dropout(inputs=h7, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h8 = tf.layers.dense(dropout_h7, N_INPUTS // 2, activation=tf.nn.relu6)
    h9 = tf.layers.dense(h8, N_INPUTS // 2, activation=tf.nn.relu)
    dropout_h9 = tf.layers.dropout(inputs=h9, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    h10 = tf.layers.dense(dropout_h9, N_INPUTS // 2, activation=tf.nn.relu6)
    h11 = tf.layers.dense(h10, N_INPUTS // 2, activation=tf.nn.relu)
    h12 = tf.layers.dense(h11, N_INPUTS // 2, activation=tf.nn.relu6)
    
    
    predictions = tf.layers.dense(h12, 1, activation=None)  # (?, 1)
    return predictions


# 2-layer RNN
def rnn2_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'state' is now a tuple containing the final state of each cell layer
    # we use state[1] below to extract the final state of the final layer
    
    # 3. pass rnn output through a dense layer
    h1 = tf.layers.dense(state[1], cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h1, 1, activation=None)  # (?, 1)
    return predictions


# create N-1 predictions
def rnnN_model(features, mode, params):
    # dynamic_rnn needs 3D shape: [BATCH_SIZE, N_INPUTS, 1]
    x = tf.reshape(features[TIMESERIES_COL], [-1, N_INPUTS, 1])

    # 2. configure the RNN
    cell1 = tf.nn.rnn_cell.GRUCell(N_INPUTS * 2)
    cell2 = tf.nn.rnn_cell.GRUCell(N_INPUTS // 2)
    cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])
    outputs, state = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)
    # 'outputs' contains the state of the final layer for every time step
    # not just the last time step (?,N_INPUTS, final cell size)
    
    # 3. pass state for each time step through a DNN, to get a prediction
    # for each time step 
    h1 = tf.layers.dense(outputs, cells.output_size, activation=tf.nn.relu)
    h2 = tf.layers.dense(h1, cells.output_size // 2, activation=tf.nn.relu)
    predictions = tf.layers.dense(h2, 1, activation=None)  # (?, N_INPUTS, 1)
    predictions = tf.reshape(predictions, [-1, N_INPUTS])
    return predictions # return prediction for each time step


# read data and convert to needed format
def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(row):
            # row is a string tensor containing the contents of one row
            features = tf.decode_csv(row, record_defaults=DEFAULTS)  # string tensor -> list of 50 rank 0 float tensors
            label = features.pop()  # remove last feature and use as label
            features = tf.stack(features)  # list of rank 0 tensors -> single rank 1 tensor
            return {TIMESERIES_COL: features}, label

        # Create list of file names that match "glob" pattern (i.e. data_file_*.csv)
        dataset = tf.data.Dataset.list_files(filename)
        # Read in data from files
        dataset = dataset.flat_map(tf.data.TextLineDataset)
        # Parse text lines as comma-separated values (CSV)
        dataset = dataset.map(decode_csv)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None  # loop indefinitely
            dataset = dataset.shuffle(buffer_size=10 * batch_size)
        else:
            num_epochs = 1  # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()

    return _input_fn


def serving_input_fn():
    feature_placeholders = {
        TIMESERIES_COL: tf.placeholder(tf.float32, [None, N_INPUTS])
    }

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    features[TIMESERIES_COL] = tf.squeeze(features[TIMESERIES_COL], axis=[2])

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)


def compute_errors(features, labels, predictions):
    labels = tf.expand_dims(labels, -1)  # rank 1 -> rank 2 to match rank of predictions

    if predictions.shape[1] == 1:
        loss = tf.losses.mean_squared_error(labels, predictions)
        rmse = tf.metrics.root_mean_squared_error(labels, predictions)
        return loss, rmse
    else:
        # one prediction for every input in sequence
        # get 1-N of (x + label)
        labelsN = tf.concat([features[TIMESERIES_COL], labels], axis=1)
        labelsN = labelsN[:, 1:]
        # loss is computed from the last 1/3 of the series
        N = (2 * N_INPUTS) // 3
        loss = tf.losses.mean_squared_error(labelsN[:, N:], predictions[:, N:])
        # rmse is computed from last prediction and last label
        lastPred = predictions[:, -1]
        rmse = tf.metrics.root_mean_squared_error(labels, lastPred)
        return loss, rmse

# RMSE when predicting same as last value
def same_as_last_benchmark(features, labels):
    predictions = features[TIMESERIES_COL][:,-1] # last value in input sequence
    return tf.metrics.root_mean_squared_error(labels, predictions)


# create the inference model
def sequence_regressor(features, labels, mode, params):
    # 1. run the appropriate model
    model_functions = {
        'linear': linear_model,
        'dnn': dnn_model,
        'cnn': cnn_model,
        'rnn': rnn_model,
        'rnn2': rnn2_model,
        'rnnN': rnnN_model}
    model_function = model_functions[params['model']]
    predictions = model_function(features, mode, params)

    # 2. loss function, training/eval ops
    loss = None
    train_op = None
    eval_metric_ops = None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        loss, rmse = compute_errors(features, labels, predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            # this is needed for batch normalization, but has no effect otherwise
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                # 2b. set up training operation
                train_op = tf.contrib.layers.optimize_loss(
                    loss,
                    tf.train.get_global_step(),
                    learning_rate=params['learning_rate'],
                    optimizer="Adam")

        # 2c. eval metric
        eval_metric_ops = {
            "RMSE": rmse,
            "RMSE_same_as_last": same_as_last_benchmark(features, labels),
        }

    # 3. Create predictions
    if predictions.shape[1] != 1:
        predictions = predictions[:, -1]  # last predicted value
    predictions_dict = {"predicted": predictions}

    # 4. return EstimatorSpec
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions_dict,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops,
        export_outputs={
            'predictions': tf.estimator.export.PredictOutput(predictions_dict)}
    )


def train_and_evaluate(output_dir, hparams):
    get_train = read_dataset(hparams['train_data_path'],
                             tf.estimator.ModeKeys.TRAIN,
                             hparams['train_batch_size'])
    get_valid = read_dataset(hparams['eval_data_path'],
                             tf.estimator.ModeKeys.EVAL,
                             1000)
    estimator = tf.estimator.Estimator(model_fn=sequence_regressor,
                                       params=hparams,
                                       config=tf.estimator.RunConfig(
                                           save_checkpoints_secs=hparams['min_eval_frequency']),
                                       model_dir=output_dir)
    train_spec = tf.estimator.TrainSpec(input_fn=get_train,
                                        max_steps=hparams['train_steps'])
    exporter = tf.estimator.LatestExporter('exporter', serving_input_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn=get_valid,
                                      steps=None,
                                      exporters=exporter,
                                      start_delay_secs=hparams['eval_delay_secs'],
                                      throttle_secs=hparams['min_eval_frequency'])
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/model.py


In [18]:
%%writefile /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Example implementation of code to run on the Cloud ML service.
"""

import traceback
import argparse
import json
import os
from . import model

if __name__ == '__main__':
  parser = argparse.ArgumentParser()
  # Input Arguments
  parser.add_argument(
      '--train_data_path',
      help='GCS or local path to training data',
      required=True
  )
  parser.add_argument(
      '--eval_data_path',
      help='GCS or local path to evaluation data',
      required=True
  )
  parser.add_argument(
      '--train_batch_size',
      help='Batch size for training steps',
      type=int,
      default=100
  )
  parser.add_argument(
      '--learning_rate',
      help='Initial learning rate for training',
      type=float,
      default=0.01
  )
  parser.add_argument(
      '--train_steps',
      help="""\
      Steps to run the training job for. A step is one batch-size,\
      """,
      type=int,
      default=0
  )
  parser.add_argument(
      '--sequence_length',
      help="""\
      This model works with fixed length sequences. 1-(N-1) are inputs, last is output
      """,
      type=int,
      default=10
  )
  parser.add_argument(
      '--output_dir',
      help='GCS location to write checkpoints and export models',
      required=True
  )
  model_names = [name.replace('_model','') \
                   for name in dir(model) \
                     if name.endswith('_model')]
  parser.add_argument(
      '--model',
      help='Type of model. Supported types are {}'.format(model_names),
      required=True
  )
  parser.add_argument(
      '--job-dir',
      help='this model ignores this field, but it is required by gcloud',
      default='junk'
  )
  parser.add_argument(
      '--eval_delay_secs',
      help='How long to wait before running first evaluation',
      default=10,
      type=int
  )
  parser.add_argument(
      '--min_eval_frequency',
      help='Minimum number of training steps between evaluations',
      default=60,
      type=int
  )

  args = parser.parse_args()
  hparams = args.__dict__
  
  # unused args provided by service
  hparams.pop('job_dir', None)
  hparams.pop('job-dir', None)

  output_dir = hparams.pop('output_dir')

  # Append trial_id to path if we are doing hptuning
  # This code can be removed if you are not using hyperparameter tuning
  output_dir = os.path.join(
      output_dir,
      json.loads(
          os.environ.get('TF_CONFIG', '{}')
      ).get('task', {}).get('trial', '')
  )

  # calculate train_steps if not provided
  if hparams['train_steps'] < 1:
     # 1,000 steps at batch_size of 100
     hparams['train_steps'] = (1000 * 100) // hparams['train_batch_size']
     print ("Training for {} steps".format(hparams['train_steps']))

  model.init(hparams)

  # Run the training job
  model.train_and_evaluate(output_dir, hparams)


Writing /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer/task.py


In [19]:
! pwd
! ls
#!{PWD}/CNN_W2P_model/trainer

## /content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trainer

/content/datalab/energy_forcasing/Sarah
BIG_q.ipynb		CNN_W2P_model  Energy_ts       train-1.csv  valid-1.csv
Build_model_task.ipynb	data	       sarah_data.csv  trained


## run the model locally on the vm

In [20]:

%%bash

DATADIR=$(pwd)/data/CNN_W2P_model 
OUTDIR=$(pwd)/CNN_W2P_model/trained
SEQ_LEN=50
rm -rf $OUTDIR
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/CNN_W2P_model/trainer \
   -- \
   --train_data_path="${DATADIR}/train-1.csv" \
   --eval_data_path="${DATADIR}/valid-1.csv"  \
   --output_dir=${OUTDIR} \
   --model=rnn --train_steps=10  --sequence_length=$SEQ_LEN

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/train-1.csv', u'--eval_data_path=/content/datalab/energy_forcasing/Sarah/data/CNN_W2P_model/valid-1.csv', u'--output_dir=/content/datalab/energy_forcasing/Sarah/CNN_W2P_model/trained', u'--model=rnn', u'--train_steps=10', u'--sequence_length=50'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 60, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <te

In [21]:
import shutil
shutil.rmtree('/data/CNN_W2P_model ', ignore_errors=True)
#os.makedirs('/data/CNN_W2P_model/')
np.random.seed(1) # makes data generation reproducible
for i in xrange(0,10):
  to_csv('data/CNN_W2P_model/train-{}.csv'.format(i), N = 1000, J = 50)  # 1000 sequences
  to_csv('data/CNN_W2P_model/valid-{}.csv'.format(i), N = 1500, J = 50)

In [22]:
%bash
gsutil -m rm -rf gs://${BUCKET}/CNN_W2P_model/*
gsutil -m cp data/CNN_W2P_model/*.csv gs://${BUCKET}/CNN_W2P_model

Removing gs://sarah-bucket/CNN_W2P_model/train-0.csv#1537315065269601...
Removing gs://sarah-bucket/CNN_W2P_model/train-1.csv#1537315065272998...
Removing gs://sarah-bucket/CNN_W2P_model/train-2.csv#1537315065301245...
Removing gs://sarah-bucket/CNN_W2P_model/train-3.csv#1537315065321115...
Removing gs://sarah-bucket/CNN_W2P_model/train-4.csv#1537315065293515...
Removing gs://sarah-bucket/CNN_W2P_model/train-5.csv#1537315065507120...
Removing gs://sarah-bucket/CNN_W2P_model/train-6.csv#1537315065553875...
Removing gs://sarah-bucket/CNN_W2P_model/train-7.csv#1537315065561190...
Removing gs://sarah-bucket/CNN_W2P_model/train-8.csv#1537315065624586...
Removing gs://sarah-bucket/CNN_W2P_model/train-9.csv#1537315065598871...
Removing gs://sarah-bucket/CNN_W2P_model/valid-0.csv#1537315065347337...
Removing gs://sarah-bucket/CNN_W2P_model/valid-1.csv#1537315065388091...
Removing gs://sarah-bucket/CNN_W2P_model/valid-2.csv#1537315065394675...
Removing gs://sarah-bucket/CNN_W2P_model/valid-3.cs

### Push out to the cloud

In [23]:
%%bash
for MODEL in linear dnn cnn rnn rnn2 rnnN; do
  OUTDIR=gs://${BUCKET}/CNN_W2P_model/${MODEL}
  JOBNAME=CNN_W2P_model_${MODEL}$(date -u +%y%m%d_%H%M%S)
  SEQ_LEN=50
  REGION=us-central1
  gsutil -m rm -rf $OUTDIR
  echo $JOBNAME
  gcloud ml-engine jobs submit training $JOBNAME \
     --region=$REGION \
     --module-name=trainer.task \
     --package-path=${PWD}/CNN_W2P_model/trainer \
     --job-dir=$OUTDIR \
     --scale-tier=PREMIUM_1 \
     --runtime-version=$TFVERSION \
     -- \
     --train_data_path="gs://${BUCKET}/CNN_W2P_model/train-1.csv" \
     --eval_data_path="gs://${BUCKET}/CNN_W2P_model/valid-1.csv"  \
     --output_dir=$OUTDIR \
     --train_steps=800000 --sequence_length=$SEQ_LEN --model=$MODEL
done


CNN_W2P_model_linear180919_013341
jobId: CNN_W2P_model_linear180919_013341
state: QUEUED
CNN_W2P_model_dnn180919_013345
jobId: CNN_W2P_model_dnn180919_013345
state: QUEUED
CNN_W2P_model_cnn180919_013349
jobId: CNN_W2P_model_cnn180919_013349
state: QUEUED
CNN_W2P_model_rnn180919_013352
jobId: CNN_W2P_model_rnn180919_013352
state: QUEUED
CNN_W2P_model_rnn2180919_013356
jobId: CNN_W2P_model_rnn2180919_013356
state: QUEUED
CNN_W2P_model_rnnN180919_013400
jobId: CNN_W2P_model_rnnN180919_013400
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_linear180919_013341] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN_W2P_model_linear180919_013341

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs CNN_W2P_model_linear180919_013341
CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_dnn180919_013345] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe CNN_W2P_model_dnn180919_013345

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs CNN_W2P_model_dnn180919_013345
CommandException: 1 files/objects could not be removed.
Job [CNN_W2P_model_cnn180919_013349] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs desc

In [24]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/CNN_W2P_model'.format(BUCKET))

TensorBoard was started successfully with pid 65703. Click here to access it.

65703

In [25]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 65703


In [26]:
print('tensorboard --logdir=gs://{}/CNN_W2P_model --port=8083'.format(BUCKET))

tensorboard --logdir=gs://sarah-bucket/CNN_W2P_model --port=8083


In [14]:
%bash
gsutil ls gs://${BUCKET}/CNN_W2P_model/cnn/export/exporter/

gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321090/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321149/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321268/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321329/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321392/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321510/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321571/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321632/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321752/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321872/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537321992/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537322053/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537322173/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537322292/
gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter

In [15]:
%bash
MODEL_NAME="CNN_W2P_model_cnn"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/CNN_W2P_model/cnn/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Deleting and deploying CNN_W2P_model_cnn ml_on_gcp from gs://sarah-bucket/CNN_W2P_model/cnn/export/exporter/1537322581/ ... this will take a few minutes


ERROR: (gcloud.ml-engine.models.create) Resource in project [qwiklabs-gcp-aebfb78fe0f1b1d1] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Creating version (this might take a few minutes)......
...................................................................................................done.


In [17]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME="CNN_W2P_model_cnn"
MODEL_VERSION="ml_on_gcp"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
  'instances': [
    {
      'key': 'u1',
      'price': 50
    },
  ]
}

In [20]:

response = requests.post(api, json=data, headers=headers)
print response.content

{"error": "Prediction failed: Expected tensor name: price, got tensor name: [u'price', u'key']."}
